In [1]:
import pandas as pd
import torch
import copy
import os

from sklearn.model_selection import train_test_split
from itertools import product
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.decomposition import PCA

from torch.utils.tensorboard import SummaryWriter

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
#import dataset
df = pd.read_csv('dataset-ml-25m/dataset.csv')

In [4]:
# Dividi il dataset in feature e target
X = df.drop(['rating'], axis=1).to_numpy()
y = df['rating'].to_numpy()

# Dividi il dataset in training, validation e test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)

y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)
#count the numebr of x_train 
print("Number of train set: ", X_train.shape[0])
print("Numebr of test set: ", X_test.shape[0])
print("Number of validation set: ", X_val.shape[0])


Number of train set:  9946
Numebr of test set:  2764
Number of validation set:  1106


In [5]:
# Applica la riduzione della dimensionalità con PCA
pca = PCA(n_components=0.95)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_val = pca.transform(X_val)
X_test = pca.transform(X_test)

In [6]:
batch_sizes = [8, 16, 32]
n_epochs = [200]
# Dimension of the prediction  layer
n_d = [8, 16, 32]
#Dimension of the attention  layer
n_a = [8, 16, 32]
#Number of successive steps in the network
n_steps = [3, 5, 7]
# Number of independent GLU layer in each GLU block
n_indipendent = [2, 3]

params = list(product(batch_sizes, n_epochs, n_d, n_a, n_steps, n_indipendent))
comb = len(batch_sizes)*len(n_d)*len(n_a) *len(n_steps) *len(n_indipendent)
print("Number of combinations: ", comb)

Number of combinations:  162


In [7]:
def get_model(batch_size, n_epochs, n_d, n_a, n_steps, n_indipendent):
    model = TabNetRegressor(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        n_independent=n_indipendent
    )
    return model

In [8]:
best_mse = float('inf')
best_model = None
best_params = None
iter = 0
for b, n_e, n_d, n_a, n_s, n_i in params:
    iter += 1
    print(f'\nIteration {iter}/{comb}')
    print(f"Configuration batch size: {b} - epochs: {n_e} - n_d: {n_d} - n_a: {n_a} - steps: {n_s} - n_indipendent: {n_i}")
    
    model = get_model(b, n_e, n_d, n_a, n_s, n_i)
    #save results for each iteration with tensorboard
    log = f'bach_size_{b}_nEpochs_{n_e}_nd_{n_d}_na_{n_a}_nSteps_{n_s}_nIndipendent_{n_i}'

    if os.path.exists("./results/TabNet/pca/"+log):
        print("Model already trained. Skipping...")
        continue

    writer = SummaryWriter(f'results/TabNet/pca/bach_size_{b}_nEpochs_{n_e}_nd_{n_d}_na_{n_a}_nSteps_{n_s}_nIndipendent_{n_i}')
    #fit model
    model.fit(
        X_train=X_train,
        y_train=y_train,
        eval_set=[(X_val, y_val)],
        eval_name=['mse'],
        patience=10,
        batch_size=b
    )

    # evaluate model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
   #save hparams for each iteration with tensorboard
    writer.add_hparams(
        {'batch_size': b, 'n_epochs': n_e, 'n_d': n_d, 'n_a': n_a, 'n_steps': n_s, 'n_indipendent': n_i},
        {'hparam/mse': mse, 'hparam/r2': r2}
    )
    print('MSE:', mse)
    print('R2 Score:', r2)
    if mse < best_mse:
        best_mse = mse
        best_model = copy.deepcopy(model)
        best_params = (b, n_e, n_d, n_a, n_s, n_i)
        print('Best model updated')
    writer.close()
            


Iteration 1/162
Configuration batch size: 8 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 3 - n_indipendent: 2
Model already trained. Skipping...

Iteration 2/162
Configuration batch size: 8 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 3 - n_indipendent: 3
Model already trained. Skipping...

Iteration 3/162
Configuration batch size: 8 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 5 - n_indipendent: 2
Model already trained. Skipping...

Iteration 4/162
Configuration batch size: 8 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 5 - n_indipendent: 3
Model already trained. Skipping...

Iteration 5/162
Configuration batch size: 8 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 7 - n_indipendent: 2
Model already trained. Skipping...

Iteration 6/162
Configuration batch size: 8 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 7 - n_indipendent: 3
Model already trained. Skipping...

Iteration 7/162
Configuration batch size: 8 - epochs: 200 - n_d: 8 - n_a: 16 - steps: 3 - n_indipendent: 2
Model already trained. Skipping...

Iter

/usr/local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.79384 | mse_mse: 0.24436 |  0:00:14s
epoch 1  | loss: 0.28454 | mse_mse: 0.23164 |  0:00:30s
epoch 2  | loss: 0.26362 | mse_mse: 0.23012 |  0:00:51s
epoch 3  | loss: 0.25912 | mse_mse: 0.24612 |  0:01:32s
epoch 4  | loss: 0.24846 | mse_mse: 0.22886 |  0:01:51s
epoch 5  | loss: 0.24583 | mse_mse: 0.21812 |  0:02:08s
epoch 6  | loss: 0.23901 | mse_mse: 0.22911 |  0:02:25s
epoch 7  | loss: 0.17796 | mse_mse: 0.10796 |  0:02:42s
epoch 8  | loss: 0.11928 | mse_mse: 0.07136 |  0:02:59s
epoch 9  | loss: 0.07523 | mse_mse: 0.0356  |  0:03:17s
epoch 10 | loss: 0.04812 | mse_mse: 0.03651 |  0:03:34s
epoch 11 | loss: 0.03895 | mse_mse: 0.01936 |  0:03:52s
epoch 12 | loss: 0.04461 | mse_mse: 0.01927 |  0:04:09s
epoch 13 | loss: 0.0358  | mse_mse: 0.02157 |  0:04:23s
epoch 14 | loss: 0.03574 | mse_mse: 0.01514 |  0:04:41s
epoch 15 | loss: 0.03203 | mse_mse: 0.03801 |  0:04:57s
epoch 16 | loss: 0.03513 | mse_mse: 0.01637 |  0:05:14s
epoch 17 | loss: 0.03107 | mse_mse: 0.01946 |  0

/usr/local/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE: 0.008039436009024499
R2 Score: 0.9637317738903414
Best model updated

Iteration 126/162
Configuration batch size: 32 - epochs: 200 - n_d: 8 - n_a: 32 - steps: 7 - n_indipendent: 3


...

MSE: 0.008882590708238332
R2 Score: 0.959928058649349

Iteration 162/162
Configuration batch size: 32 - epochs: 200 - n_d: 32 - n_a: 32 - steps: 7 - n_indipendent: 3


/usr/local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.0339  | mse_mse: 0.2507  |  0:00:15s
epoch 1  | loss: 0.28588 | mse_mse: 0.22611 |  0:00:31s
epoch 2  | loss: 0.25186 | mse_mse: 0.29915 |  0:00:48s
epoch 3  | loss: 0.19435 | mse_mse: 0.12115 |  0:01:05s
epoch 4  | loss: 0.12441 | mse_mse: 0.12056 |  0:01:21s
epoch 5  | loss: 0.10885 | mse_mse: 0.08292 |  0:01:38s
epoch 6  | loss: 0.10242 | mse_mse: 0.08825 |  0:01:55s
epoch 7  | loss: 0.10641 | mse_mse: 0.08532 |  0:02:12s
epoch 8  | loss: 0.10254 | mse_mse: 0.07656 |  0:02:28s
epoch 9  | loss: 0.0978  | mse_mse: 0.1192  |  0:02:45s
epoch 10 | loss: 0.09648 | mse_mse: 0.07826 |  0:03:02s
epoch 11 | loss: 0.09355 | mse_mse: 0.07616 |  0:03:19s
epoch 12 | loss: 0.0949  | mse_mse: 0.10214 |  0:03:36s
epoch 13 | loss: 0.08558 | mse_mse: 0.06609 |  0:03:52s
epoch 14 | loss: 0.08423 | mse_mse: 0.06691 |  0:04:09s
epoch 15 | loss: 0.07519 | mse_mse: 0.08354 |  0:04:26s
epoch 16 | loss: 0.06156 | mse_mse: 0.04374 |  0:04:43s
epoch 17 | loss: 0.04862 | mse_mse: 0.06675 |  0

/usr/local/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE: 0.012078978372291535
R2 Score: 0.9455082274069742


In [11]:
#pint the best model
print(best_model)
print("Best MSE: ", best_mse)
print("R2 score", r2_score(y_test, best_model.predict(X_test)))


TabNetRegressor(n_d=32, n_a=16, n_steps=3, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=1, n_independent=3, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.001, seed=0, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, scheduler_params={}, mask_type='sparsemax', input_dim=552, output_dim=1, device_name='auto', n_shared_decoder=1, n_indep_decoder=1)
Best MSE:  0.00727536348501948
R2 score 0.9671787265165809


In [12]:
#save the best model in a file csv
best_model.save_model('best_model_tabnet_pca.csv')

Successfully saved model at best_model_tabnet_pca.csv.zip


'best_model_tabnet_pca.csv.zip'